In [ ]:
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

In [ ]:
RUTA_XLSX_SNR = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoJuridico\2.Submodelo_Insumos_SNR\tbl_submodelo_insumos_snr_lote_4.xlsx"
SHEET_NAME = "tbl_submodeloinsumosnr_lote_4"
TIPOS_DATOS = {'codigo_municipio':str,
    'snrderecho_codigo_naturaleza_juridica':str,
    'snrtitular_numero_documento':str,
    'predioregistro_codigo_orip':str,
    'predioregistro_matricula_inmobiliaria':str,
    'predioregistro_numero_predial_nuevo_en_fmi':str,
    'predioregistro_numero_predial_anterior_en_fmi':str,
    'snrfuentederecho_numero_documento':str}

df_data_SNR = pd.read_excel(RUTA_XLSX_SNR, sheet_name=SHEET_NAME, dtype = TIPOS_DATOS)

<h3>Insumo: XTF SNR</h3>

In [ ]:
sheet_name = 'CONTROL_DE_CAMBIOS'
sheet_id = '1n_t9qtwLeIklH5UFoH9iDPiAa6uN4HZeR1-atsKTYeU'

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

control_cambios_esig_original = pd.read_csv(url)
columnas_seleccion = ['terreno_codigo', 
    'Nombre', 
    'EDICION',
    'POSIBLE SERVIDUMBRE', 
    'FECHA EDITOR',
    'OBSERVACIONES EDITOR',
    'REVISADO',
    'FECHA REVISOR',
    'OBSERVACIONES REVISOR',
    'ESTADO DE EDICION']

control_cambios_esig_filtro = control_cambios_esig_original[columnas_seleccion]

control_cambios_esig_filtro = control_cambios_esig_filtro.copy()

control_cambios_esig_filtro.loc[control_cambios_esig_filtro['EDICION'] == 'REQUIERE VISITA DE CAMPO', 'EDICION'] = 'Requiere visita campo'
control_cambios_esig_filtro.loc[control_cambios_esig_filtro['EDICION'] == 'SI, (EDICION GEOGRAFICA)', 'EDICION'] = 'Con edición geo'
control_cambios_esig_filtro.loc[control_cambios_esig_filtro['EDICION'] == 'NO, (NO REQUIERE AJUSTE GEOGRAFICO)', 'EDICION'] = 'No requiere ajuste geo'

<h3>Cruce Gestión Cambios VS SNR XTF</h3>

In [ ]:
# ** Terrenos Gestionados SIG Espacializados
df_snr_gestion_sig = pd.merge(df_data_SNR, control_cambios_esig_filtro, left_on='predioregistro_numero_predial_nuevo_en_fmi', right_on='terreno_codigo', how='inner')

df_snr_gestion_sig

<h3>Insumos: Registro 1</h3>

In [ ]:
ruta_registro_1 = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
tabla_registro_1_consolidada = "Con_REGISTRO1_ENE2024"

registro_1 = os.path.join(ruta_registro_1, tabla_registro_1_consolidada)

df_registro_1 = pd.DataFrame.spatial.from_table(registro_1)

In [ ]:

df_registro_1_gestion_sig = pd.merge(df_registro_1, control_cambios_esig_filtro, left_on='NUMERO_PREDIAL', right_on='terreno_codigo', how='left')

<h3>Exportación DF Gestión Catastro - Registro 1</h3>

In [ ]:
ruta_salida_export_registro_1_gestion_catastro = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoJuridico\4.Gestion_Cambios_Equipo_SIG"
nombre_salida_csv = "Registro_1_Vs_Gestion_Cambios_SIG.csv"

directorio_salida = os.path.join(ruta_salida_export_registro_1_gestion_catastro, nombre_salida_csv)

df_registro_1_gestion_sig.to_csv(directorio_salida)
print(f"Se exporta {nombre_salida_csv} correctamente")

<h3>Espacialización de Resultados</h3>

<h4>Preparación de la capa Terrenos</h4>

In [ ]:
ruta_filebd = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
ruta_shp = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoSIG\1.Geo_Gestion_Cambios\zShps"

CAPA_TERRENOS_R = 'Analitica_UT_Consolidada\R_TERRENO'
CAPA_TERRENOS_U = 'Analitica_UT_Consolidada\\U_TERRENO'
SHP = '_Gestion_Cambios_SIG.shp'

capa_terrenos_rurales = os.path.join(ruta_filebd, CAPA_TERRENOS_R)
capa_terrenos_urbanos = os.path.join(ruta_filebd, CAPA_TERRENOS_U)

# ** Terrenos Rurales
df_terreno_rural = pd.DataFrame.spatial.from_featureclass(capa_terrenos_rurales)
df_terreno_rural.head()
# ** Terrenos Urbanos
df_terreno_urbano = pd.DataFrame.spatial.from_featureclass(capa_terrenos_urbanos)

# ** Selección de columnas
columnas_selecciones = ['CODIGO',
                        'CODIGO_ANTERIOR',
                        'SHAPE']

# ** DF Terrenos Rurales, Urbanos
df_terreno_rural_reducido = df_terreno_rural[columnas_selecciones]
df_terreno_urbano_reducido = df_terreno_urbano[columnas_selecciones]

# ** Concatenación de dataframes
df_terrenos = pd.concat([df_terreno_rural_reducido, df_terreno_urbano_reducido])

<h2>Cruce Espacial Registro 1 - Gestión Cambios - Terrenos</h2>

In [ ]:

df_terreno_registro_1_gestion_sig = pd.merge(df_terrenos, df_registro_1_gestion_sig, left_on='CODIGO', right_on='NUMERO_PREDIAL', how='inner')

NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'terrenos_registro_1_control_de_cambios_esig'.upper()}"
ruta_salida_terrenos_registro_1_control_cambios = os.path.join(ruta_filebd, NOMBRE_CAPA_SALIDA)

df_terreno_registro_1_gestion_sig.spatial.to_featureclass(location=ruta_salida_terrenos_registro_1_control_cambios)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")